In [1]:
%load_ext autoreload
%autoreload 3

In [2]:
import json
import os
from pathlib import Path

import pandas as pd
from sklearn.metrics import precision_score, recall_score, accuracy_score, roc_auc_score, confusion_matrix
from splink import block_on, DuckDBAPI, Linker, SettingsCreator
from splink.blocking_analysis import count_comparisons_from_blocking_rule, cumulative_comparisons_to_be_scored_from_blocking_rules_chart, n_largest_blocks
import splink.comparison_library as cl
import splink.comparison_level_library as cll
from splink.exploratory import completeness_chart, profile_columns
from upath import UPath

from mozilla_sec_eia.models.sec10k.utils.cloud import GCSArchive, convert_ex21_id_to_filename
from mozilla_sec_eia.models.sec_eia_record_linkage.preprocessing import add_sec_company_id_to_subsidiaries, prepare_sec10k_basic_info_df, prepare_eia_df, prepare_ex21_df

# Inputs

### EIA

TODO: materialize asset and read in from Dagster GCS storage

In [13]:
from mozilla_sec_eia.models.sec_eia_record_linkage.create_eia_input import get_eia_utilities_table

In [14]:
eia_df = get_eia_utilities_table()

In [28]:
eia_df.head(1)

,utility_id_eia,utility_id_pudl,utility_name_eia,report_date,street_address,city,state,zip_code,plants_reported_owner,plants_reported_operator,plants_reported_asset_manager,plants_reported_other_relationship,entity_type,attention_line,address_2,zip_code_4,contact_firstname,contact_lastname,contact_title,phone_number,phone_extension,contact_firstname_2,contact_lastname_2,contact_title_2,phone_number_2,phone_extension_2,data_maturity
0,66550,16573.0,Telyon AMZ Windsor LLC,2024-01-01,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,monthly_update


### SEC 10K Basic Info

TODO: read in asset from Dagster GCS storage

In [24]:
sec_path = UPath("gs://sec10k-outputs/v2/basic_10k_company_info")

In [25]:
raw_sec_df = pd.DataFrame()
for file in sec_path.iterdir():
    if file.name.split(".")[-1] == "parquet":
        raw_sec_df = pd.concat([raw_sec_df, pd.read_parquet(sec_path / file.name)])

In [26]:
raw_sec_df = raw_sec_df.reset_index().pivot_table(values="value", index="filename", columns="key", aggfunc="first")
raw_sec_df.columns.name = None

In [29]:
raw_sec_df.head(1)

,]fiscal_year_end,]irs_number,]state_of_incorporation,business_phone,central_index_key,city,company_conformed_name,date_of_name_change,film_number,fiscal_year_end,form_type,former_conformed_name,irs_number,organization_name,sec_act,sec_file_number,standard_industrial_classification,state,state_of_incorporation,street_1,street_2,zip
filename,,,,,,,,,,,,,,,,,,,,,,
edgar/data/1000015/0000912057-00-014793.txt,NaN,NaN,NaN,2039736700,0001000015,stamford,meta group inc,NaN,585471,1231,10-k,NaN,060971675,NaN,NaN,000-27280,"services-engineering, accounting, research, ma...",ct,de,208 harbor dr,NaN,06912-0061


In [15]:
sec_clean_df = prepare_sec10k_basic_info_df(raw_sec_df)

In [21]:
sec_asset_df = pd.read_parquet(UPath("gs://sec10k-outputs/v2/out_sec_10k__parents_and_subsidiaries/2023q1.parquet"))

### Ex. 21

TODO: get rid of this section

In [17]:
ex21_path = UPath("gs://sec10k-outputs/v2/ex21_company_ownership_info")

In [18]:
raw_ex21_df = pd.DataFrame()
for file in ex21_path.iterdir():
    if file.name.split(".")[-1] == "parquet":
        year_quarter_df = pd.read_parquet(ex21_path / file.name)
        report_year = file.name[:4]
        year_quarter_df.loc[:, "report_year"] = report_year
        year_quarter_df.loc[:, "report_year"] = pd.to_datetime(year_quarter_df["report_year"], format="%Y").dt.year
        raw_ex21_df = pd.concat([raw_ex21_df, year_quarter_df])

# Preprocess Ex. 21

TODO: get rid of this section

In [19]:
ex21_clean_df = prepare_ex21_df(raw_ex21_df)

/Users/katielamb/CatalystCoop/mozilla-sec-eia/src/mozilla_sec_eia/models/sec_eia_record_linkage/preprocessing.py:168: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df = df.fillna(np.nan)


In [34]:
sec_match_df = sec_clean_df.drop_duplicates(subset=["central_index_key", "company_name", "loc_of_incorporation", "report_year"])
merged_df = sec_match_df.merge(ex21_clean_df, how="inner", on="company_name", suffixes=("_sec", "_ex21"))
merged_df.loc[:, "loc_tokens_sec"] = merged_df["loc_of_incorporation_sec"].fillna("").str.lower().str.split()
merged_df.loc[:, "loc_tokens_ex21"] = merged_df["loc_of_incorporation_ex21"].fillna("").str.lower().str.split()
merged_df["loc_overlap"] = merged_df.apply(
    lambda row: len(set(row["loc_tokens_sec"]) & set(row["loc_tokens_ex21"])), axis=1
)
merged_df["report_year_diff"] = merged_df.apply(
    lambda row: abs(int(row["report_year_sec"]) - int(row["report_year_ex21"])), axis=1
)
# Sort by CIK, company_name, loc_overlap, and report_year_diff
# so that we can then choose the first record in each CIK, company_name group
merged_df = merged_df.sort_values(by=["central_index_key", "company_name", "loc_overlap", "report_year_diff"],
                                  ascending=[True, True, False, True]
                                 )
# Select the row with the highest loc overlap and nearest report years for each CIK and company name
cik_and_company_pairs = merged_df.groupby(["central_index_key", "company_name"], as_index=False).first()
# We now have the closest matching CIK and company name pairs
# We want to get the best matching CIK for each company name and loc of incorporation
# Select the row with the highest loc overlap and nearest report years for each company name and loc pair
cik_and_company_pairs = cik_and_company_pairs.sort_values(by=["company_name", "loc_of_incorporation_ex21", "loc_overlap", "report_year_diff"],
                                                          ascending=[True, True, False, True]
                                                         )
closest_match = cik_and_company_pairs.groupby(["company_name", "loc_of_incorporation_ex21"], as_index=False).first()
closest_match = closest_match.drop_duplicates(subset=["central_index_key", "company_name", "loc_of_incorporation_ex21"])

In [35]:
# a company name and location of incorporation should match to only one CIK
closest_match.duplicated(subset=["company_name", "loc_of_incorporation_ex21"]).value_counts()

False    5808
Name: count, dtype: int64

In [36]:
# it's okay if there's duplication here
# multiple subsidiaries can point to the same CIK
# and company names can change and they still keep the same CIK
closest_match.central_index_key.duplicated().value_counts()

central_index_key
False    5532
True      276
Name: count, dtype: int64

In [37]:
ex21_with_cik = ex21_clean_df.merge(
    closest_match[["company_name", "central_index_key", "loc_of_incorporation_ex21"]].rename(columns={"loc_of_incorporation_ex21": "loc_of_incorporation"}),
    how="left",
    on=["company_name", "loc_of_incorporation"],
).rename(columns={"central_index_key": "subsidiary_cik"})

In [38]:
ex21_with_cik.subsidiary_cik.isnull().value_counts()

subsidiary_cik
True     2900030
False      21674
Name: count, dtype: int64

In [39]:
ex21_with_cik = ex21_with_cik.merge(closest_match[["company_name", "central_index_key"]],
                                    how="left",
                                    on="company_name"
                                   ).rename(columns={"central_index_key": "company_name_merge_cik"})

In [40]:
# if a subsidiary doesn't have a CIK and has a null location
# but its company name was assigned a CIK (with a different location)
# then assign that CIK to the subsidiary
ex21_with_cik["subsidiary_cik"] = ex21_with_cik["subsidiary_cik"].where(
    ~(ex21_with_cik.subsidiary_cik.isnull()) | ~(ex21_with_cik.loc_of_incorporation.isnull()), 
    ex21_with_cik["company_name_merge_cik"]
)

In [41]:
# there should be fewer null CIKs now
ex21_with_cik.subsidiary_cik.isnull().value_counts()

subsidiary_cik
True     2897527
False      24221
Name: count, dtype: int64

In [42]:
archive = GCSArchive()
md = archive.get_metadata()

In [43]:
ex21_with_cik.loc[:, "filename"] = convert_ex21_id_to_filename(ex21_with_cik)

In [44]:
ex21_with_cik = ex21_with_cik.merge(md["cik"],
                                    how="left",
                                    left_on="filename",
                                    right_index=True).rename(columns={"cik": "parent_cik"})

In [45]:
ex21_with_cik = add_sec_company_id_to_subsidiaries(ex21_with_cik)

In [46]:
# remove the Ex. 21 subsidiaries who were matched to a filing company
unmatched_ex21_df = ex21_with_cik[ex21_with_cik.subsidiary_cik.isnull()]

# Preprocess SEC and EIA

Does it actually make sense to add in the Ex. 21 subsidiaries when we only have company name?
Does it make more sense to do a direct match on company name after
the SEC basic info to EIA match is done? And if there's a conflicting SEC match (one basic info and one Ex. 21) then review it manually?

TODO: get rid of these cells

TODO: filter for only "files_10k" filers

In [67]:
# find a way to use state of incorporation even though it's not on the EIA side?
sec_full_clean_df = pd.concat([sec_clean_df, 
                               unmatched_ex21_df[["sec_company_id", "report_year", "company_name", "company_name_no_legal", "company_name_mphone", "state_of_incorporation"]]
                              ])

In [71]:
sec_full_clean_df = sec_full_clean_df.reset_index(drop=True).reset_index(names="record_id")

In [24]:
# for now, just use sec_clean_df without Ex. 21 subsidiaries
sec_clean_df = sec_clean_df.reset_index(drop=True).reset_index(names="record_id")

In [15]:
eia_clean_df = prepare_eia_df(eia_df)

In [27]:
sec_clean_df.record_id.is_unique

True

In [28]:
eia_clean_df.record_id.is_unique

True

In [30]:
# TODO: move this into preprocessing
# strip legal terms and then make a list column from company name
# use this for blocking and comnparison levels
eia_clean_df.loc[:, "company_name_mphone_list"] = eia_clean_df["company_name_mphone"].str.split()

In [31]:
sec_clean_df.loc[:, "company_name_mphone_list"] = sec_clean_df["company_name_mphone"].str.split()

In [32]:
eia_clean_df.loc[:, "zip_code"] = eia_clean_df["zip_code"].str[:5]

In [33]:
sec_clean_df.loc[:, "zip_code"] = sec_clean_df["zip_code"].str[:5]

In [34]:
# I think we don't need this column
eia_clean_df.loc[:, "street_address_list"] = eia_clean_df["street_address"].str.split()
sec_clean_df.loc[:, "street_address_list"] = sec_clean_df["street_address"].str.split()

TODO: import from config file

In [36]:
SHARED_COLS = [
    "record_id",
    "report_date",
    "report_year",
    "company_name",
    "company_name_no_legal",
    "street_address",
    "street_address_list",
    "street_address_2",
    "city",
    "state",  # could use state of incorporation from SEC
    "zip_code",
    "phone_number",
    "company_name_mphone",
    "company_name_mphone_list"
]

In [55]:
eia_match_df = eia_clean_df[SHARED_COLS]

In [38]:
sec_match_df = sec_clean_df[SHARED_COLS]

TODO: import from config

In [43]:
match_cols = ["company_name", "state", "city", "street_address", "zip_code"]

In [49]:
# duplicates exist because of differing report years
eia_match_df.duplicated(subset=match_cols).value_counts()

True     138441
False     39407
Name: count, dtype: int64

In [52]:
sec_match_df.duplicated(subset=match_cols).value_counts()

True     168445
False     64515
Name: count, dtype: int64

In [56]:
sec_match_df = sec_match_df.sort_values(by="report_year", ascending=False).drop_duplicates(subset=match_cols, keep="first")

In [57]:
eia_match_df = eia_match_df.sort_values(by="report_year", ascending=False).drop_duplicates(subset=match_cols, keep="first")

In [39]:
# could try to use keywords like gas, electricity, utility etc.
sec_clean_df["standard_industrial_classification"].value_counts().head(5)

standard_industrial_classification
asset-backed securities [6189]          20311
pharmaceutical preparations [2834]       8530
state commercial banks [6022]            7886
real estate investment trusts [6798]     7706
services-prepackaged software [7372]     6007
Name: count, dtype: int64

# Link SEC and EIA

## Exploratory Analysis

In [128]:
db_api = DuckDBAPI()

In [129]:
# this goes way down when we start matching in the Ex. 21 subsidiaries
completeness_chart(sec_match_df, db_api=db_api)

alt.LayerChart(...)

In [130]:
completeness_chart(eia_match_df, db_api=db_api)

alt.LayerChart(...)

In [131]:
profile_columns(sec_match_df[match_cols], db_api=DuckDBAPI(), top_n=10, bottom_n=5)

alt.VConcatChart(...)

In [132]:
profile_columns(eia_match_df[match_cols], db_api=DuckDBAPI(), top_n=10, bottom_n=5)

alt.VConcatChart(...)

## Blocking

TODO: import BLOCKING RULES from config

In [67]:
counts = count_comparisons_from_blocking_rule(
    table_or_tables=[sec_match_df, eia_match_df],
    blocking_rule=BLOCKING_RULES[0],
    link_type="link_only",
    unique_id_column_name='record_id',
    db_api=db_api,
)

counts

{'number_of_comparisons_generated_pre_filter_conditions': 988101,
 'number_of_comparisons_to_be_scored_post_filter_conditions': 988101,
 'filter_conditions_identified': '',
 'equi_join_conditions_identified': 'SUBSTRING(l.company_name_mphone, 1, 4) = SUBSTRING(r.company_name_mphone, 1, 4)',
 'link_type_join_condition': 'where l."source_dataset" || \'-__-\' || l."record_id" < r."source_dataset" || \'-__-\' || r."record_id" and l."source_dataset" != r."source_dataset"'}

In [68]:
result = n_largest_blocks(
    table_or_tables=[sec_match_df, eia_match_df],
    blocking_rule=BLOCKING_RULES[0],
    link_type="link_only",
    db_api=db_api,
    n_largest=3
)

result.as_pandas_dataframe()

,key_0,count_l,count_r,block_count
0,AMRK,888,85,75480
1,INTR,468,157,73476
2,FRST,836,82,68552


In [69]:
cumulative_comparisons_to_be_scored_from_blocking_rules_chart(
    table_or_tables=[sec_match_df, eia_match_df],
    blocking_rules=BLOCKING_RULES,
    db_api=db_api,
    unique_id_column_name='record_id',
    link_type="link_only",
)

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

alt.Chart(...)

## Create Model

TODO: import comparisons from config

In [334]:
# NOT USED
company_name_comparison = cl.CustomComparison(
    comparison_levels = [
        cll.NullLevel("company_name"),
        cll.ExactMatchLevel("company_name"),
        # cll.ExactMatchLevel("company_name_no_legal"),
        # cll.LevenshteinLevel("company_name", distance_threshold=1),
        cll.JaroWinklerLevel("company_name_no_legal", distance_threshold=.95),
        # cll.ArraySubsetLevel("company_name_mphone_list"),
        cll.ArrayIntersectLevel("company_name_mphone_list", min_intersection=3)
    ],
    output_column_name="company_name",
    comparison_description=None
)

In [386]:
company_name_comparison = cl.NameComparison(
    "company_name_no_legal",
    jaro_winkler_thresholds=[.95],
)
"""
company_name_comparison = cl.JaccardAtThresholds(
     "company_name",
    # dmeta_col_name="company_name_mphone_list" # this was breaking it for some reason
)
"""
print(company_name_comparison.get_comparison("duckdb").human_readable_description)

Comparison 'NameComparison' of "company_name_no_legal".
Similarity is assessed using the following ComparisonLevels:
    - 'company_name_no_legal is NULL' with SQL rule: "company_name_no_legal_l" IS NULL OR "company_name_no_legal_r" IS NULL
    - 'Exact match on company_name_no_legal' with SQL rule: "company_name_no_legal_l" = "company_name_no_legal_r"
    - 'Jaro-Winkler distance of company_name_no_legal >= 0.95' with SQL rule: jaro_winkler_similarity("company_name_no_legal_l", "company_name_no_legal_r") >= 0.95
    - 'All other comparisons' with SQL rule: ELSE



In [449]:
address_comparison = cl.LevenshteinAtThresholds(
    "street_address",
    distance_threshold_or_thresholds=[1]
).configure(term_frequency_adjustments=True)
print(address_comparison.get_comparison("duckdb").human_readable_description)

Comparison 'LevenshteinAtThresholds' of "street_address".
Similarity is assessed using the following ComparisonLevels:
    - 'street_address is NULL' with SQL rule: "street_address_l" IS NULL OR "street_address_r" IS NULL
    - 'Exact match on street_address' with SQL rule: "street_address_l" = "street_address_r"
    - 'Levenshtein distance of street_address <= 1' with SQL rule: levenshtein("street_address_l", "street_address_r") <= 1
    - 'All other comparisons' with SQL rule: ELSE



In [422]:
# NOT USED
address_comparison = cl.CustomComparison(
    comparison_levels = [
        cll.NullLevel("street_address"),
        cll.ExactMatchLevel("street_address"),
        cll.LevenshteinLevel("street_address", distance_threshold=1),
        cll.ArraySubsetLevel("street_address_list"),
    ],
    output_column_name="street_address",
    comparison_description=None
)

In [388]:
# Use state and city instead of zip code
zip_code_comparison = cl.ExactMatch("zip_code").configure(term_frequency_adjustments=True)

In [450]:
state_comparison = cl.ExactMatch("state").configure(term_frequency_adjustments=True)

In [451]:
city_comparison = cl.NameComparison(
    "city",
    jaro_winkler_thresholds=[0.9]
)
print(city_comparison.get_comparison("duckdb").human_readable_description)

Comparison 'NameComparison' of "city".
Similarity is assessed using the following ComparisonLevels:
    - 'city is NULL' with SQL rule: "city_l" IS NULL OR "city_r" IS NULL
    - 'Exact match on city' with SQL rule: "city_l" = "city_r"
    - 'Jaro-Winkler distance of city >= 0.9' with SQL rule: jaro_winkler_similarity("city_l", "city_r") >= 0.9
    - 'All other comparisons' with SQL rule: ELSE



In [452]:
settings = SettingsCreator(
    link_type="link_only",
    unique_id_column_name="record_id",
    comparisons=[
        company_name_comparison,
        address_comparison,
        # zip_code_comparison,
        state_comparison,
        city_comparison
    ],
    blocking_rules_to_generate_predictions=[
        BLOCKING_RULES
    ],
    retain_intermediate_calculation_columns=True,
)

linker = Linker([sec_match_df, eia_match_df], settings, db_api=DuckDBAPI())

TODO: import deterministic rules

In [453]:
deterministic_rules = [
    block_on("company_name_mphone", "company_name_mphone"),
    # block_on("street_address"),
    "jaro_winkler_similarity(r.company_name, l.company_name) >= .95 and l.city = r.city",
    # "substr(l.company_name_mphone,1,4) = substr(r.company_name_mphone,1,4) and l.city = r.city and jaccard(r.street_address, l.street_address) >= .9",
    "substr(l.company_name_mphone,1,4) = substr(r.company_name_mphone,1,4) and l.city = r.city and l.street_address = r.street_address",
]

linker.training.estimate_probability_two_random_records_match(deterministic_rules, recall=0.95)

Probability two random records match is estimated to be  1.78e-06.
This means that amongst all possible pairwise record comparisons, one in 562,858.42 are expected to match.  With 2,542,342,605 total possible comparisons, we expect a total of around 4,516.84 matching pairs


In [ ]:
linker.training.estimate_u_using_random_sampling(max_pairs=1e8)

----- Estimating u probabilities using random sampling -----


In [427]:
training_blocking_rule = block_on("company_name", "company_name")
training_session_fname_sname = (
    linker.training.estimate_parameters_using_expectation_maximisation(training_blocking_rule)
)


----- Starting EM training session -----

Estimating the m probabilities of the model by blocking on:
(l."company_name" = r."company_name") AND (l."company_name" = r."company_name")

Parameter estimates will be made for the following comparison(s):
    - company_name_no_legal
    - street_address
    - state
    - city

Parameter estimates cannot be made for the following comparison(s) since they are used in the blocking rules: 

Level Jaro-Winkler distance of company_name_no_legal >= 0.95 on comparison company_name_no_legal not observed in dataset, unable to train m value

Level All other comparisons on comparison company_name_no_legal not observed in dataset, unable to train m value

Iteration 1: Largest change in params was -0.347 in the m_probability of city, level `All other comparisons`
Iteration 2: Largest change in params was 0.307 in the m_probability of city, level `All other comparisons`
Iteration 3: Largest change in params was 0.0403 in the m_probability of city, level `A

In [428]:
training_blocking_rule = block_on("street_address", "street_address")
training_session_fname_sname = (
    linker.training.estimate_parameters_using_expectation_maximisation(training_blocking_rule)
)


----- Starting EM training session -----

Estimating the m probabilities of the model by blocking on:
(l."street_address" = r."street_address") AND (l."street_address" = r."street_address")

Parameter estimates will be made for the following comparison(s):
    - company_name_no_legal
    - state
    - city

Parameter estimates cannot be made for the following comparison(s) since they are used in the blocking rules: 
    - street_address

Iteration 1: Largest change in params was -0.395 in the m_probability of city, level `All other comparisons`
Iteration 2: Largest change in params was 0.889 in the m_probability of company_name_no_legal, level `All other comparisons`
Iteration 3: Largest change in params was 0.285 in probability_two_random_records_match
Iteration 4: Largest change in params was 0.0152 in probability_two_random_records_match
Iteration 5: Largest change in params was 0.048 in the m_probability of city, level `All other comparisons`
Iteration 6: Largest change in params 

In [429]:
linker.visualisations.match_weights_chart()

alt.VConcatChart(...)

In [430]:
linker.visualisations.m_u_parameters_chart()

alt.HConcatChart(...)

In [420]:
settings = linker.misc.save_model_to_json(
    "model_unsupervised_city_state_0.json", overwrite=True
)

## Make Predictions

In [431]:
# it's helpful to keep threshold at .5 just to see what makes it into blocking
# df_predictions = linker.inference.predict(threshold_match_probability=0.5)
df_predictions = linker.inference.predict()

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

Blocking time: 9.73 seconds
Predict time: 0.52 seconds


In [432]:
preds_df = df_predictions.as_pandas_dataframe()

In [433]:
preds_df.sort_values(by="match_probability")

,match_weight,match_probability,source_dataset_l,source_dataset_r,record_id_l,record_id_r,company_name_no_legal_l,company_name_no_legal_r,gamma_company_name_no_legal,tf_company_name_no_legal_l,tf_company_name_no_legal_r,bf_company_name_no_legal,bf_tf_adj_company_name_no_legal,street_address_l,street_address_r,street_address_list_l,street_address_list_r,gamma_street_address,bf_street_address,state_l,state_r,gamma_state,tf_state_l,tf_state_r,bf_state,bf_tf_adj_state,city_l,city_r,gamma_city,tf_city_l,tf_city_r,bf_city,bf_tf_adj_city,company_name_mphone_l,company_name_mphone_r,match_key
32260,-24.047823,5.766122e-08,__splink__input_table_0,__splink__input_table_1,82087,113663,sutro biopharma,stirling energy systems solar one,0,0.000019,0.000029,0.985981,1.0,"310 utah ave., suite 150",suite 150,"[310, utah, ave.,, suite, 150]","[suite, 150]",0.0,0.265921,ca,az,0,0.149142,0.012950,0.310698,1.0,south san francisco,phoenix,0,0.001438,0.003511,0.398403,1.0,STR BFRM,STRLNK ENRJ SSTMS SLR ON,3
27875,-24.047823,5.766122e-08,__splink__input_table_0,__splink__input_table_1,126035,113797,corner growth acquisition 2,grubb and ellis management services,0,0.000010,0.000019,0.985981,1.0,"251 lytton avenue, suite 200",suite 200,"[251, lytton, avenue,, suite, 200]","[suite, 200]",0.0,0.265921,ca,pa,0,0.149142,0.030197,0.310698,1.0,palo alto,pittsburgh,0,0.001850,0.003656,0.398403,1.0,KRNR KR0 AKKSXN,KRB ANT ELS MNJMNT SRFSS,3
27993,-24.047823,5.766122e-08,__splink__input_table_0,__splink__input_table_1,125096,97905,altus power,allegheny ridge wind farm,0,0.000010,0.000038,0.985981,1.0,"2200 atlantic street, 6th floor",6th floor,"[2200, atlantic, street,, 6th, floor]","[6th, floor]",0.0,0.265921,ct,ca,0,0.020325,0.149142,0.310698,1.0,stamford,san francisco,0,0.003789,0.013374,0.398403,1.0,ALTS PWR,ALKHN RJ WNT FRM,3
28003,-24.047823,5.766122e-08,__splink__input_table_0,__splink__input_table_1,115402,91508,clearway energy,clipper windpower,0,0.000038,0.000029,0.985981,1.0,"300 carnegie center, suite 300",suite 300,"[300, carnegie, center,, suite, 300]","[suite, 300]",0.0,0.265921,nj,ca,0,0.031159,0.149142,0.310698,1.0,princeton,carpinteria,0,0.002118,0.000189,0.398403,1.0,KLRW ENRJ,KLPR WNTPWR,3
28024,-24.047823,5.766122e-08,__splink__input_table_0,__splink__input_table_1,125009,77758,benchmark 2020 b21 mortgage trust,bountiful city city of,0,0.000010,0.000048,0.985981,1.0,200 west street,198 south 200 west street,"[200, west, street]","[198, south, 200, west, street]",0.0,0.265921,ny,ut,0,0.113010,0.010475,0.310698,1.0,new york,bountiful city,0,0.086944,0.000022,0.398403,1.0,BNXMRK B MRTKJ TRST,BNTFL ST ST OF,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1038434,NaN,NaN,__splink__input_table_0,__splink__input_table_1,137784,70294,farmer brothers,farmers electric ia,0,0.000029,0.000038,0.985981,1.0,20333 s normandie ave,"1959 yoder ave,sw","[20333, s, normandie, ave]","[1959, yoder, ave,sw]",NaN,NaN,ca,ia,0,0.149142,0.016527,0.310698,1.0,torrance,kalona,0,0.002485,0.000011,0.398403,1.0,FRMR BR0RS,FRMRS ELKTRK I,0
1038441,NaN,NaN,__splink__input_table_0,__splink__input_table_1,139631,137540,international game technology,intergen north america,0,0.000048,0.000029,0.985981,1.0,6355 south buffalo drive,4th floor,"[6355, south, buffalo, drive]","[4th, floor]",NaN,NaN,nv,ma,0,0.019288,0.041401,0.310698,1.0,las vegas,burlington,0,0.010477,0.001415,0.398403,1.0,INTRNXNL KM TXNLJ,INTRJN NR0 AMRK,0
1038443,NaN,NaN,__splink__input_table_0,__splink__input_table_1,90853,13424,monster arts,minnesota solar csg 4,0,0.000010,0.000029,0.985981,1.0,806 east avenida pico,"200 wellington street west, su","[806, east, avenida, pico]","[200, wellington, street, west,, su]",NaN,NaN,ca,None,-1,0.149142,NaN,1.000000,1.0,san clemente,toronto,0,0.000346,0.002129,0.398403,1.0,MNSTR ARTS,MNST SLR KSK,0
1038454,NaN,NaN,__splink__input_table_0,__splink__input_table_1,108136,195

In [434]:
# join on utility_id_eia and CIK
preds_validation_df = preds_df.merge(sec_clean_df[["record_id", "central_index_key", "company_name_raw"]],
                                     how="left",
                                     left_on="record_id_l",
                                     right_on="record_id")

In [435]:
preds_validation_df = preds_validation_df.merge(eia_clean_df[["record_id", "utility_id_eia"]],
                                                how="left",
                                                left_on="record_id_r",
                                                right_on="record_id")

In [436]:
preds_validation_df = preds_validation_df.sort_values(
    by=["central_index_key", "utility_id_eia", "match_probability"], ascending=False
).drop_duplicates(subset=["central_index_key", "utility_id_eia"], keep="first")

In [437]:
preds_validation_df[preds_validation_df.match_probability > .9]

,match_weight,match_probability,source_dataset_l,source_dataset_r,record_id_l,record_id_r,company_name_no_legal_l,company_name_no_legal_r,gamma_company_name_no_legal,tf_company_name_no_legal_l,tf_company_name_no_legal_r,bf_company_name_no_legal,bf_tf_adj_company_name_no_legal,street_address_l,street_address_r,street_address_list_l,street_address_list_r,gamma_street_address,bf_street_address,state_l,state_r,gamma_state,tf_state_l,tf_state_r,bf_state,bf_tf_adj_state,city_l,city_r,gamma_city,tf_city_l,tf_city_r,bf_city,bf_tf_adj_city,company_name_mphone_l,company_name_mphone_r,match_key,record_id_x,central_index_key,company_name_raw,record_id_y,utility_id_eia
889845,5.679807,0.980865,__splink__input_table_0,__splink__input_table_1,51956,22658,constellation energy,constellation newenergy,1,0.000029,0.000077,6085.754919,1.000000,1310 point street,None,"[1310, point, street]",NaN,-1.0,1.000000,md,md,1,0.023298,0.023298,14.856341,2.034020,baltimore,baltimore,2,0.003678,0.003678,94.80739,1.654881,KNSTLXN ENRJ,KNSTLXN NWNRJ,0,51956,0001868275,constellation energy corp,22658,58491
884109,13.095633,0.999886,__splink__input_table_0,__splink__input_table_1,120267,96849,evergy,evergy,2,0.000019,0.000019,872345.689655,0.059564,1200 main street,1200 main street,"[1200, main, street]","[1200, main, street]",2.0,5.407499,mo,mo,1,0.011744,0.011744,14.856341,4.035057,kansas city,kansas city,2,0.001973,0.001973,94.80739,3.085372,EFRJ,EFRJ,0,120267,0001711269,"evergy, inc.",96849,64428
893941,12.486567,0.999826,__splink__input_table_0,__splink__input_table_1,120222,96211,consol energy,consol energy,2,0.000058,0.000058,872345.689655,0.019855,275 technology drive,275 technology drive,"[275, technology, drive]","[275, technology, drive]",2.0,5.407499,pa,pa,1,0.030197,0.030197,14.856341,1.569346,canonsburg,canonsburg,2,0.000390,0.000390,94.80739,15.603165,KNSL ENRJ,KNSL ENRJ,0,120222,0001710366,consol energy inc.,96211,4299
943594,9.161274,0.998256,__splink__input_table_0,__splink__input_table_1,119271,83669,vistra energy,vistra energy,2,0.000019,0.000019,872345.689655,0.059564,6555 sierra drive,6555 sierra drive,"[6555, sierra, drive]","[6555, sierra, drive]",2.0,5.407499,tx,tx,1,0.080866,0.080866,14.856341,0.586015,irving,irving,2,0.004380,0.004380,94.80739,1.389595,FSTR ENRJ,FSTR ENRJ,0,119271,0001692819,vistra energy corp.,83669,62723
860414,7.576311,0.994788,__splink__input_table_0,__splink__input_table_1,119274,71441,vistra,vistra,2,0.000058,0.000058,872345.689655,0.019855,6555 sierra drive,6555 sierra drive,"[6555, sierra, drive]","[6555, sierra, drive]",2.0,5.407499,tx,tx,1,0.080866,0.080866,14.856341,0.586015,irving,irving,2,0.004380,0.004380,94.80739,1.389595,FSTR,FSTR,0,119274,0001692819,vistra corp.,71441,5504
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1026765,12.087133,0.999770,__splink__input_table_0,__splink__input_table_1,153106,79761,archer daniels midland,archer daniels midland,2,0.000058,0.000058,872345.689655,0.019855,4666 faries pkwy,4666 faries pkwy,"[4666, faries, pkwy]","[4666, faries, pkwy]",2.0,5.407499,il,il,1,0.033191,0.033191,14.856341,1.427770,decatur,decatur,2,0.000468,0.000468,94.80739,13.002638,ARXR TNLS MTLNT,ARXR TNLS MTLNT,0,153106,0000007084,archer daniels midland co,79761,772
656833,9.809977,0.998887,__splink__input_table_0,__splink__input_table_1,150546,79913,appalachian power,appalachian power,2,0.000077,0.000077,872345.689655,0.014891,1 riverside plaza,1 riverside plaza,"[1, riverside, plaza]","[1, riverside, plaza]",2.0,5.407499,oh,oh,1,0.018770,0.018770,14.856341,2.524754,columbus,columbus,2,0.003009,0.003009,94.80739,2.022633,APLXN PWR,APLXN PWR,0,150546,0000006879,appalachian power co,79913,733
640747,10.888046,0.999473,__splink__input_table_0,__splink__input_table_1,144743,80319,american crystal sugar /mn/,american crystal sugar,1,0.000010,0.000029,6085.754919,1.000000,101 n 3rd st,No

In [89]:
validation_df = pd.read_csv("sec_eia_validation_set.csv", dtype={"central_index_key": str})

In [90]:
validation_df["central_index_key"] = validation_df["central_index_key"].str.zfill(10)

In [438]:
merged_df = validation_df.merge(
    preds_validation_df[["record_id_l", "record_id_r", "central_index_key", "utility_id_eia", "match_probability", "gamma_company_name_no_legal"]].drop_duplicates(keep="first"),
    how="left",
    on=["central_index_key", "utility_id_eia"],
    indicator=True
)

In [439]:
merged_df["predicted_match"] = merged_df["_merge"].map({"both": 1, "left_only": 0})

In [440]:
merged_df["predicted_match"] = merged_df["predicted_match"].where(
    (merged_df.match_probability > .95),
    0
)

In [441]:
merged_df.head(50)

,central_index_key,utility_id_eia,sec_company_name,eia_company_name,match,record_id_l,record_id_r,match_probability,gamma_company_name_no_legal,_merge,predicted_match
0,0000003153,195,alabama power co,NaN,1,2575,80977,0.999018,2,both,1.0
1,0001868941,58702,"fluence energy, inc.",Fluence,0,126809,21615,0.000002,0,both,0.0
2,0000041091,7140,georgia power co,NaN,1,50428,68242,0.029853,2,both,0.0
3,0000022198,4062,columbus southern power co /oh/,Columbus Southern Power Co,1,129635,96300,0.997628,1,both,1.0
4,0001326160,5416,duke energy corp,NaN,1,37661,71555,0.926352,2,both,0.0
5,0000030371,54905,"duke energy carolinas, llc",Duke Energy Carolinas LLC,1,133261,118543,0.987916,2,both,1.0
6,0000869446,57140,berkshire realty co inc /de,Berkshire Wind Power Cooperative Corp,0,198821,89415,0.000030,0,both,0.0
7,0000092122,18195,southern co,southern co services inc,0,50417,111824,0.000063,0,both,0.0
8,0000092122,17650,southern co,Southern Power Co,0,50417,49613,0.004315,0,both,0.0
9,0000075488,14328,pacific gas & electric co,NaN,1,2898,55480,0.624991,2,both,0.0


In [442]:
precision = precision_score(merged_df['match'], merged_df['predicted_match'])
recall = recall_score(merged_df['match'], merged_df['predicted_match'])
accuracy = accuracy_score(merged_df['match'], merged_df['predicted_match'])
# roc_auc = roc_auc_score(merged_df['match'], merged_df['match_probability'])

# Confusion matrix
conf_matrix = confusion_matrix(merged_df['match'], merged_df['predicted_match'])

In [443]:
precision, recall, accuracy

(np.float64(0.8888888888888888),
 np.float64(0.6153846153846154),
 0.7142857142857143)

In [444]:
pd.DataFrame(
    conf_matrix,
    index=["Negative", "Positive"],
    columns=["Predicted Negative", "Predicted Positive"]
)

,Predicted Negative,Predicted Positive
Negative,7,1
Positive,5,8


In [445]:
incorrect_df = merged_df[merged_df.match != merged_df.predicted_match]

In [446]:
incorrect_df

,central_index_key,utility_id_eia,sec_company_name,eia_company_name,match,record_id_l,record_id_r,match_probability,gamma_company_name_no_legal,_merge,predicted_match
2,0000041091,7140,georgia power co,NaN,1,50428,68242,0.029853,2,both,0.0
4,0001326160,5416,duke energy corp,NaN,1,37661,71555,0.926352,2,both,0.0
9,0000075488,14328,pacific gas & electric co,NaN,1,2898,55480,0.624991,2,both,0.0
10,0001031296,6526,firstenergy corp,FirstEnergy,0,14192,69716,0.999707,2,both,1.0
13,0001031296,32208,firstenergy corp,First Energy Corp,1,14192,121855,0.010697,1,both,0.0
20,0000018675,3266,central maine power co,NaN,1,126771,176663,0.897700,2,both,0.0


In [447]:
recs_to_view = []
for idx, rec in incorrect_df.iterrows():
    full_rec = preds_validation_df[
        (preds_validation_df.record_id_l == rec.record_id_l) & 
        (preds_validation_df.record_id_r == rec.record_id_r)
    ].squeeze()
    if full_rec.empty:
        continue
    recs_to_view.append(full_rec.to_dict())

In [448]:
linker.visualisations.waterfall_chart(recs_to_view, filter_nulls=True)

alt.LayerChart(...)

# Save good predictions

In [192]:
preds_df[preds_df.match_probability >= .95].sort_values(by="match_probability")

,match_weight,match_probability,source_dataset_l,source_dataset_r,record_id_l,record_id_r,company_name_l,company_name_r,gamma_company_name,tf_company_name_l,tf_company_name_r,bf_company_name,bf_tf_adj_company_name,street_address_l,street_address_r,gamma_street_address,tf_street_address_l,tf_street_address_r,bf_street_address,bf_tf_adj_street_address,zip_code_l,zip_code_r,gamma_zip_code,tf_zip_code_l,tf_zip_code_r,bf_zip_code,bf_tf_adj_zip_code,city_l,city_r,gamma_city,tf_city_l,tf_city_r,bf_city,bf_tf_adj_city,company_name_mphone_l,company_name_mphone_r,street_address_list_l,street_address_list_r,match_key
199607,4.265490,0.950575,__splink__input_table_0,__splink__input_table_1,20077,117512,prt group incorporated,pratt and whitney power systems,0,0.000019,0.000010,0.991220,1.000000,80 lamberton rd,mail stop 191-13,0,0.000036,0.000012,0.865948,1.000000,06095,06095,1,0.000191,0.000191,1148.002189,3.403266,windsor,windsor,2,0.000279,0.000279,126.999683,24.882561,PRT KRP,PRT ANT HTN PWR SSTMS,"[80, lamberton, rd]","[mail, stop, 191-13]",0
12041,4.277468,0.950964,__splink__input_table_0,__splink__input_table_1,219453,113555,cogentrix energy incorporated,green country energy limited liability company,0,0.000019,0.000038,0.991220,1.000000,9405 arrowpoint blvd,9405 arrowpoint blvd,2,0.000534,0.000534,14580.390627,0.015600,28273,28273,1,0.001256,0.001256,1148.002189,0.516567,charlotte,chalotte,1,0.014155,0.000022,79.923487,1.000000,KJNTRKS ENRJ,KRN KNTR ENRJ,"[9405, arrowpoint, blvd]","[9405, arrowpoint, blvd]",1
12805,4.277468,0.950964,__splink__input_table_0,__splink__input_table_1,219453,115755,cogentrix energy incorporated,jackson county power limited liability company,0,0.000019,0.000029,0.991220,1.000000,9405 arrowpoint blvd,9405 arrowpoint blvd,2,0.000534,0.000534,14580.390627,0.015600,28273,28273,1,0.001256,0.001256,1148.002189,0.516567,charlotte,chaarlotte,1,0.014155,0.000011,79.923487,1.000000,KJNTRKS ENRJ,JKSN KNT PWR,"[9405, arrowpoint, blvd]","[9405, arrowpoint, blvd]",1
8137,4.278093,0.950984,__splink__input_table_0,__splink__input_table_1,64813,3879,rand logistics incorporated,norridgewock river road solar limited liabilit...,0,0.000029,0.000019,0.991220,1.000000,333 washington street,333 washington street,2,0.001056,0.001056,14580.390627,0.007888,07302,07302,1,0.002332,0.002332,1148.002189,0.278152,jersey city,jersey city,2,0.002998,0.002998,126.999683,2.312506,RNT LJSTKS,NRJWK RFR RT SLR,"[333, washington, street]","[333, washington, street]",1
8136,4.278093,0.950984,__splink__input_table_0,__splink__input_table_1,64813,5193,rand logistics incorporated,anderson solar farm limited liability company,0,0.000029,0.000029,0.991220,1.000000,333 washington street,333 washington street,2,0.001056,0.001056,14580.390627,0.007888,07302,07302,1,0.002332,0.002332,1148.002189,0.278152,jersey city,jersey city,2,0.002998,0.002998,126.999683,2.312506,RNT LJSTKS,ANTRSN SLR FRM,"[333, washington, street]","[333, washington, street]",1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
199278,27.514584,1.000000,__splink__input_table_0,__splink__input_table_1,27759,142183,diamond brands incorporated,diamond brands incorporated,2,0.000029,0.000029,7612.680596,0.037986,1800 cloquet avenue,1800 cloquet avenue,2,0.000036,0.000036,14580.390627,0.233998,55720,55720,1,0.000078,0.000078,1148.002189,8.265075,cloquet,cloquet,2,0.000078,0.000078,126.999683,88.866289,TMNT BRNTS,TMNT BRNTS,"[1800, cloquet, avenue]","[1800, cloquet, avenue]",0
485070,27.655362,1.000000,__splink__input_table_0,__splink__input_table_1,50420,95697,gulf power company,gulf power company,2,0.000038,0.000038,7612.680596,0.028490,one energy place,one energy place,2,0.000024,0.000024,14580.390627,0.350997,32520,32520,1,0.000056,0.000056,1148.002189,11.571104,pensacola,pensacola,2,0.000111,0.000111,126.999683,62.206402,KLF PWR,KLF PWR,"[one, energy, place]","[one, energy, 

# Match to Ex. 21 subsidiaries

In [38]:
# match EIA records that don't have a prediction to EIA subsidiaries
# can reuse code from SEC module?